In [1]:
import yaml
import torch
import matplotlib.pyplot as plt

In [2]:
config_file = './configs/lego.yaml'
with open(config_file, 'r') as file:
    config = yaml.safe_load(file)

In [3]:
from utils.DataLoaderLego import DataLoaderLego

data_loader = DataLoaderLego(config['data'])

images, poses, focal = data_loader.getDataset('train')

In [4]:
from utils.Camera import Camera

cameras = []
H, W = images[0].shape[:2] # proBABLY SWAPED

for i in range(images.shape[0]):
    cameras.append(Camera(H, W, poses[i], focal))


In [5]:
from utils.Renderer import Renderer

renderer = Renderer(config['renderer'])

r1, r2 = cameras[0].getRays()
r1 = r1.expand(r2.shape)

points, dists = renderer.getSparsePoints(r1,r2)

points = points.reshape((-1,) + points.shape[2:])
dists = dists.reshape((-1, ) + dists.shape[2:])

torch.Size([800, 800, 64, 2])


In [6]:
from models.TinyNerf import TinyNerf

model = TinyNerf(6)
device = 'cpu'
if torch.cuda.is_available():
    device = torch.cuda.current_device()
model = model.to(device)


6


In [7]:
from utils.TrainerTinyNerf import TrainerTinyNerf
import imageio.v3 as iio

test_img = "nerf_synthetic/lego/val/r_0.png"
test_img = iio.imread(test_img)[:,:,:3]
test_pose = torch.tensor([
                [
                    -0.963964581489563,
                    -0.2611401677131653,
                    0.0507759265601635,
                    0.2046843022108078
                ],
                [
                    0.26603081822395325,
                    -0.9462433457374573,
                    0.18398693203926086,
                    0.7416750192642212
                ],
                [
                    7.450580596923828e-09,
                    0.1908649355173111,
                    0.9816163182258606,
                    3.957021951675415
                ],
                [
                    0.0,
                    0.0,
                    0.0,
                    1.0
                ]
            ])
trainer = TrainerTinyNerf(model, device, images, cameras, renderer, config['trainer'])
trainer.train(test_img[None,...], test_pose[None,...], focal)

6


torch.Size([800, 800, 64, 2])
4 torch.Size([800, 800, 64, 5]) torch.Size([800, 800, 64])


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 3.81 GiB total capacity; 3.67 GiB already allocated; 5.12 MiB free; 3.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF